In [1]:
from google.colab import drive
drive.mount('/content/drive')


import os
os.chdir('/content/drive/MyDrive/NLU_1012') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

 1012_proposal_modification.pdf
'An ERNIE-Based Joint Model for Chinese Named Entity Recognition.pdf'
'Chinese Named Entity Recognition Based on BERT and Lightweight Feature Extraction Model.pdf'
'Copy of NYU HPC README.gdoc'
 DebiasingNLUModelViaCausalIntervention.pdf
'EEC data'
 Experiments.gsheet
'Named entity recognition of Chinese electronic medical records based on a hybrid neural network and medical MC-BERT.pdf'
 proposal_discussion.gdoc
 Result.ipynb
 Results_Corsair.txt
 Results_RoBERTa_Twitter_NER.txt
 Results_textCNN_Twitter_NER.txt
 roberta.base
 w2v


#### Data Preprocess

In [3]:
import pandas as pd

In [4]:
eec_dt= pd.read_csv('/content/drive/MyDrive/NLU_1012/EEC data/Equity-Evaluation-Corpus.csv')

In [5]:
eec_dt.head()

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


In [6]:
eec_dt.isnull().sum()

ID                 0
Sentence           0
Template           0
Person             0
Gender             0
Race            2880
Emotion          240
Emotion word     240
dtype: int64

In [7]:
eec_dt.shape

(8640, 8)

In [8]:
final_dt= eec_dt[['Sentence', 'Person', 'Gender', 'Race', 'Emotion']].dropna()
final_dt.head()

,Sentence,Person,Gender,Race,Emotion
0,Alonzo feels angry.,Alonzo,male,African-American,anger
1,Alonzo feels furious.,Alonzo,male,African-American,anger
2,Alonzo feels irritated.,Alonzo,male,African-American,anger
3,Alonzo feels enraged.,Alonzo,male,African-American,anger
4,Alonzo feels annoyed.,Alonzo,male,African-American,anger


In [9]:
final_dt.isnull().sum()

Sentence    0
Person      0
Gender      0
Race        0
Emotion     0
dtype: int64

In [10]:
# final_dt['Emotion']= final_dt['Emotion'].map(emotion_dict)
final_dt.groupby(by=['Gender', 'Race']).Emotion.value_counts()

Gender  Race              Emotion
female  African-American  anger      350
                          fear       350
                          joy        350
                          sadness    350
        European          anger      350
                          fear       350
                          joy        350
                          sadness    350
male    African-American  anger      350
                          fear       350
                          joy        350
                          sadness    350
        European          anger      350
                          fear       350
                          joy        350
                          sadness    350
Name: Emotion, dtype: int64

In [11]:
import numpy as np

train_index= []
dev_index= []
test_index= []
 

for gender in list(final_dt['Gender'].unique()):
  for race in list(final_dt['Race'].unique()):
    for emotion in list(final_dt['Emotion'].unique()):
      index= np.array(final_dt[(final_dt['Emotion']==emotion) & (final_dt['Gender']==gender) & (final_dt['Race']==race)].index)
      np.random.shuffle(index)
      splits= np.split(index, 5)
      train_index.append(splits[:3])
      dev_index.append(splits[3])
      test_index.append(splits[4])


In [12]:
train_dt= final_dt.loc[list((np.concatenate(train_index)).flatten())].sample(frac=1)
dev_dt= final_dt.loc[list((np.concatenate(dev_index)).flatten())].sample(frac=1)
test_dt= final_dt.loc[list((np.concatenate(test_index)).flatten())].sample(frac=1)

In [13]:
with open('/content/EEC.train.jsonl', 'w') as f:
  print(train_dt.rename({'Emotion': 'label', 'Sentence': 'text'}, axis=1)[['label', 'text']]\
      .to_json(orient='records', lines=True),file=f, flush=False)

In [14]:
with open('/content/EEC.dev.jsonl', 'w') as f:
  print(dev_dt.rename({'Emotion': 'label', 'Sentence': 'text'}, axis=1)[['label', 'text']]\
      .to_json(orient='records', lines=True),file=f, flush=False)

In [ ]:
with open('/content/EEC.test.jsonl', 'w') as f:
  print(test_dt.rename({'Emotion': 'label', 'Sentence': 'text'}, axis=1)[['label', 'text']]\
      .to_json(orient='records', lines=True),file=f, flush=False)

#### Result ReadIn

In [22]:
!pip install transformers
!pip install bert_serving
!pip install bert_serving.client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import os
os.chdir('/content/drive/MyDrive/NLU_1012/EEC data') 

import pandas as pd
import pickle 
import MyData_NER
import bert_serving
import bert_serving.client
import transformers
 
with open('/content/drive/MyDrive/NLU_1012/EEC data/EEC_test_out.pkl', 'rb') as f:
  output = pickle.load(f)

In [24]:
optimal_result= output[0]
optimal_result.keys()

dict_keys(['rate', 'actual_x', 'actual_y', 'true_input', 'true_labels', 'factual_outputs', 'counterfactual_outputs'])

In [25]:
print('optimal rate: ' + str(optimal_result['rate']))

optimal rate: (-1.0999999999999999, 2.700000000000001)


In [26]:
test_factual_dict= {'text': [], 'label': list(optimal_result['actual_y'])\
            , 'true_labels': list(optimal_result['true_labels'])\
            , 'anger': [], 'fear': [], 'joy': [], 'sadness': []} 
for input in optimal_result['actual_x']:
  test_factual_dict['text'].append(' '.join(input.split('____')[:-1])+'.')

for input in optimal_result['factual_outputs']:
  test_factual_dict['anger'].append(input[0]) 
  test_factual_dict['fear'].append(input[1]) 
  test_factual_dict['joy'].append(input[2]) 
  test_factual_dict['sadness'].append(input[3]) 

test_factual_dt= pd.DataFrame(test_factual_dict)

In [27]:
test_counterfactual_dict= {'text': [], 'label': list(optimal_result['actual_y'])\
            , 'true_labels': list(optimal_result['true_labels'])\
            , 'anger': [], 'fear': [], 'joy': [], 'sadness': []}
for input in optimal_result['actual_x']:
  test_counterfactual_dict['text'].append(' '.join(input.split('____')[:-1])+'.')

for input in optimal_result['counterfactual_outputs']:
  test_counterfactual_dict['anger'].append(input[0]) 
  test_counterfactual_dict['fear'].append(input[1]) 
  test_counterfactual_dict['joy'].append(input[2]) 
  test_counterfactual_dict['sadness'].append(input[3]) 

test_counterfactual_dt= pd.DataFrame(test_counterfactual_dict)

In [28]:
# get the data
origin_test = pd.read_csv("/content/drive/MyDrive/NLU_1012/EEC data/Equity-Evaluation-Corpus.csv", delimiter=",")
# df = pd.DataFrame(sentences,
# 				  columns=["Sentence", "Gender", "Race", "Emotion", "ID", "Library", "Negative", "Positive", "Neutral",
# 						   "Compound"])
origin_test= origin_test[['Sentence', 'Person', 'Gender', 'Race', 'Emotion']].dropna()
origin_test['Sentence']= origin_test['Sentence'].str.lower() 

In [29]:
factual_test_dt = origin_test.merge(test_factual_dt, how='inner', left_on=['Sentence', 'Emotion']\
                                    , right_on=['text', 'label']).drop(columns= ['text', 'label'])
factual_test_dt.head()

,Sentence,Person,Gender,Race,Emotion,true_labels,anger,fear,joy,sadness
0,alonzo feels angry.,Alonzo,male,African-American,anger,0,0.069987,0.058677,0.219168,0.059840
1,alonzo feels excited.,Alonzo,male,African-American,joy,2,0.412180,0.336391,0.279204,0.233161
2,jamel feels angry.,Jamel,male,African-American,anger,0,0.099114,0.407212,0.269692,0.223520
3,jamel feels enraged.,Jamel,male,African-American,anger,0,0.224070,0.137152,-0.012757,0.153005
4,jamel feels devastated.,Jamel,male,African-American,sadness,3,0.063985,0.050614,0.310019,0.456985


In [30]:
counterfactual_test_dt = origin_test.merge(test_counterfactual_dt, how='inner', left_on=['Sentence', 'Emotion']\
                                    , right_on=['text', 'label']).drop(columns= ['text', 'label'])
counterfactual_test_dt.head()

,Sentence,Person,Gender,Race,Emotion,true_labels,anger,fear,joy,sadness
0,alonzo feels angry.,Alonzo,male,African-American,anger,0,-0.054659,0.119397,-0.484893,-0.581460
1,alonzo feels excited.,Alonzo,male,African-American,joy,2,-0.026366,-0.048891,-0.624591,0.239885
2,jamel feels angry.,Jamel,male,African-American,anger,0,-0.009501,0.002632,0.134219,-0.301176
3,jamel feels enraged.,Jamel,male,African-American,anger,0,-0.625134,0.357794,0.533997,-0.579853
4,jamel feels devastated.,Jamel,male,African-American,sadness,3,0.090126,0.068105,0.106768,0.259460


### T-test

In [31]:
import pandas as pd
from scipy.stats import ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt 

In [32]:
def run_t_test(category_1, category_2):
	temp_significance = []
	temp_hypothesis = []

	# significance value and degrees of freedom
	sig = 0.01/2  # (0.05/2 for two-sided test)
	# degrees of freedom = 1050
	# rejection Hypothesis area: +-1.98
	rej = 2.58 
 
	result = ttest_ind(category_1, category_2, axis=0, equal_var=1, nan_policy="propagate")
	statistic = result[0]
	pvalue = result[1]
 
	if pvalue < sig:
		temp_significance.append(0)
		temp_significance.append("Significative")
	else:
		temp_significance.append(0)
		temp_significance.append("Not Significative")

	# hypothesis rejection
	if statistic > 0 and statistic > rej:
		temp_hypothesis.append(0)
		temp_hypothesis.append("Rejected")
	elif statistic < 0 and statistic < -rej:
		temp_hypothesis.append(0)
		temp_hypothesis.append("Rejected")
	else:
		temp_hypothesis.append(0)
		temp_hypothesis.append("Confirmed")

	return (
		temp_significance,
		temp_hypothesis
	)


In [33]:
def aggregate_racial_bias_results(data):
  emotions = ["anger", "joy", "fear", "sadness"]
  genders = ["female", "male"]
  races = ["European", "African-American"]
  race_result_list = []
  race_mean_differences = []
  
  for emotion in emotions:
    for race in races:
      gender_df = data[(data["Race"] == race) & (data["Emotion"] == emotion)]
      temp_data = { 
        "Race": race,
        "Emotion": emotion,
        "Mean": gender_df[emotion].mean()
      }
      race_result_list.append(temp_data)

  racial_bias_results = pd.DataFrame(race_result_list) 

  for i in racial_bias_results.Mean:
    if len(race_mean_differences) % 2 == 0:
      race_mean_differences.append(i)
    else:
      race_mean_differences.append(i - (race_mean_differences[-1]))

  difference = []
  for i in race_mean_differences:
    if len(difference) % 2 == 0:  
      difference.append(0)
    else:
      difference.append(i)

  difference = pd.Series(difference)
  racial_bias_results = racial_bias_results.assign(Difference=difference)

  prevalence = []
  for n in racial_bias_results.Difference:
    if not len(prevalence) % 2 == 0:
      if n == 0:
        prevalence.append("EA=AA")
      else:
        if n > 0:
          prevalence.append("EA<AA")
        if n < 0:
          prevalence.append("EA>AA")
    else:
      prevalence.append(0)

  prevalence = pd.Series(prevalence)
  racial_bias_results = racial_bias_results.assign(Prevalence=prevalence)

  significance = []
  hypothesis = [] 
  for emotion in emotions:
    african_var = data.loc[(data["Race"] == "African-American") & (data["Emotion"] == emotion)]
    european_var = data.loc[(data["Race"] == "European") & (data["Emotion"] == emotion)]
    category_1 = african_var[emotion]
    category_2 = european_var[emotion]
    temp = run_t_test(category_1, category_2)
    significance.extend(temp[0])
    hypothesis.extend(temp[1])

  temp_significance = pd.Series(significance)
  racial_bias_results = racial_bias_results.assign(Significativity=temp_significance)

  temp_hypothesis = pd.Series(hypothesis)
  racial_bias_results = racial_bias_results.assign(Hypothesis=temp_hypothesis)
  
  return racial_bias_results



In [34]:
aggregate_racial_bias_results(factual_test_dt) 

,Race,Emotion,Mean,Difference,Prevalence,Significativity,Hypothesis
0,European,anger,0.146897,0.000000,0,0,0
1,African-American,anger,0.151584,0.004687,EA<AA,Not Significative,Confirmed
2,European,joy,0.149015,0.000000,0,0,0
3,African-American,joy,0.169217,0.020202,EA<AA,Not Significative,Confirmed
4,European,fear,0.051285,0.000000,0,0,0
5,African-American,fear,0.048055,-0.003230,EA>AA,Not Significative,Confirmed
6,European,sadness,0.137717,0.000000,0,0,0
7,African-American,sadness,0.132138,-0.005579,EA>AA,Not Significative,Confirmed


In [35]:
aggregate_racial_bias_results(counterfactual_test_dt)

,Race,Emotion,Mean,Difference,Prevalence,Significativity,Hypothesis
0,European,anger,-0.030516,0.000000,0,0,0
1,African-American,anger,-0.031253,-0.000737,EA>AA,Not Significative,Confirmed
2,European,joy,-0.083878,0.000000,0,0,0
3,African-American,joy,-0.103282,-0.019404,EA>AA,Not Significative,Confirmed
4,European,fear,0.051022,0.000000,0,0,0
5,African-American,fear,-0.018854,-0.069876,EA>AA,Not Significative,Confirmed
6,European,sadness,-0.064593,0.000000,0,0,0
7,African-American,sadness,-0.059390,0.005203,EA<AA,Not Significative,Confirmed


In [36]:
def aggregate_gender_bias_results(data):
	gender_result_list = []
	gender_mean_differences = []
	
	emotions = ["anger", "joy", "fear", "sadness"]
	genders = ["female", "male"]
	races = ["European", "African-American"]

	for emotion in emotions:
		for gender in genders:
			gender_df = data.loc[(data["Gender"] == gender) & (data["Emotion"] == emotion)]
			temp_data = { 
				"Gender": gender,
				"Emotion": emotion,
				"Mean": gender_df[emotion].mean()
			}
			gender_result_list.append(temp_data)

	gender_bias_results = pd.DataFrame(gender_result_list) 
 
	for i in gender_bias_results.Mean:
		if len(gender_mean_differences) % 2 == 0:
			gender_mean_differences.append(i)
		else:
			gender_mean_differences.append(i - (gender_mean_differences[-1]))

	difference = []
	for i in gender_mean_differences:
		if len(difference) % 2 == 0:
			# differences=[]
			difference.append(0)
		else:
			difference.append(i)

	difference = pd.Series(difference)
	gender_bias_results = gender_bias_results.assign(Difference=difference)
 
	prevalence = []
	for n in gender_bias_results.Difference:
		if not len(prevalence) % 2 == 0:
			if n == 0:
				prevalence.append("F=M")
			else:
				if n > 0:
					prevalence.append("F<M")
				if n < 0:
					prevalence.append("F>M")
		else:
			prevalence.append(0)

	prevalence = pd.Series(prevalence)
	gender_bias_results = gender_bias_results.assign(Prevalence=prevalence) 
 
	significance = []
	hypothesis = [] 
	for emotion in emotions:
		male_var = data.loc[(data["Gender"] == "male") & (data["Emotion"] == emotion)]
		female_var = data.loc[(data["Gender"] == "female") & (data["Emotion"] == emotion)]
		category_1 = male_var[emotion]
		category_2 = female_var[emotion]
		temp = run_t_test(category_1, category_2)
		significance.extend(temp[0])
		hypothesis.extend(temp[1])

	temp_significance = pd.Series(significance)
	gender_bias_results = gender_bias_results.assign(Significativity=temp_significance)

	temp_hypothesis = pd.Series(hypothesis)
	gender_bias_results = gender_bias_results.assign(Hypothesis=temp_hypothesis) 

	return gender_bias_results

In [37]:
aggregate_gender_bias_results(factual_test_dt)

,Gender,Emotion,Mean,Difference,Prevalence,Significativity,Hypothesis
0,female,anger,0.146049,0.000000,0,0,0
1,male,anger,0.152180,0.006132,F<M,Not Significative,Confirmed
2,female,joy,0.184507,0.000000,0,0,0
3,male,joy,0.136743,-0.047764,F>M,Not Significative,Rejected
4,female,fear,0.040730,0.000000,0,0,0
5,male,fear,0.059527,0.018797,F<M,Not Significative,Confirmed
6,female,sadness,0.143120,0.000000,0,0,0
7,male,sadness,0.127376,-0.015744,F>M,Not Significative,Confirmed


In [38]:
aggregate_gender_bias_results(counterfactual_test_dt)  

,Gender,Emotion,Mean,Difference,Prevalence,Significativity,Hypothesis
0,female,anger,-0.020314,0.000000,0,0,0
1,male,anger,-0.040722,-0.020408,F>M,Not Significative,Confirmed
2,female,joy,-0.107560,0.000000,0,0,0
3,male,joy,-0.081373,0.026187,F<M,Not Significative,Confirmed
4,female,fear,0.035909,0.000000,0,0,0
5,male,fear,-0.006353,-0.042261,F>M,Not Significative,Confirmed
6,female,sadness,-0.078095,0.000000,0,0,0
7,male,sadness,-0.047070,0.031024,F<M,Not Significative,Confirmed
